# Project DataType

Les étapes:

- lire le fichier data_for_datatype.txt
- extraire les données I/O (documents et classes)
- on contrsuit le pipeline
    - on va tester pusieurs vectorizers: CountVactorizer, TFIDFVextorizer
    - on utilise le Baysian classifier: MultinomialNB
- on verifie à l'oeil si cela fonctionne
- on fait varirer les hyperparametres pour touver le meilleur score (cross validation score)

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

NGRAM_RANGE = (1, 3)

vectorizer = CountVectorizer(
    input="content",
    analyzer="char",
    ngram_range=NGRAM_RANGE,
    stop_words=None,
    binary=False
)

TFIDFVectorizer = TfidfVectorizer(
    input="content",
    analyzer="char",
    ngram_range=NGRAM_RANGE,
    stop_words=None,
    binary=False
)

# l'entrée
raw_documents_vector = []

# la sortie
y_vector = []

file_name = "./data_for_datatype.txt"
with open(file_name, "rt") as f:
    for line in f:
        data = line.split(" ", maxsplit=1)
        y = data[0]
        raw_document = str(len(data[1])) + " " + data[1]
        
        # je garde les données dans des listes
        y_vector.append(y)
        raw_documents_vector.append(raw_document)
   
print("start")
vectorizer.fit(raw_documents_vector)
#TFIDFVectorizer.fit(raw_documents_vector)
print("done")

start
done


In [4]:
# on verifie le bon fonctionnement
tokenizer = vectorizer.build_tokenizer()
tokenizer("201804")
#tokenizer = TFIDFVectorizer.build_tokenizer()
#tokenizer("201804")

['201804']

In [5]:
# on affiche N mots du vocabulaire
i_max = 10
for i, word in enumerate(vectorizer.vocabulary_.keys()):
    if i > i_max:
        break
    else:
        print(word)

6.4
825
64

wr
is

pus
 mh
rs 
ubi
b1 
no


In [6]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

# on créé une instance du classifier
clf = MultinomialNB()
#clf = KNeighborsClassifier(algorithm="kd_tree")
pipeline_clf = Pipeline(
    [
        ("vectorizer", vectorizer),
        ("clf", clf)
    ]
)

#pipeline_TFIDF = Pipeline(
#    [
#        ("vectorizer", TFIDFVectorizer),
#        ("clf", clf)
#    ]
#)

# on divise nos donnée en 2 population: training/test
X_train, X_test, Y_train, Y_test = train_test_split(raw_documents_vector, y_vector, train_size=0.8)

print("train :", len(Y_train))
print("test :", len(Y_test))

# on entraine le classifier
pipeline_clf.fit(X_train, Y_train)
#pipeline_TFIDF.fit(X_train, Y_train)
print("done")

/home/timotheehuard/.local/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


train : 295022
test : 73756
done


In [7]:
print("start predict")
predicted = pipeline_clf.predict(X_test)
#predictedTFIDF = pipeline_TFIDF.predict(X_test)s
print("done")

start predict
done


In [8]:
import numpy as np

mean = np.mean(predicted == Y_test)
total = len(Y_test)
correct_ones = np.sum(predicted == Y_test)
print("total test :", total)
print("correct :", correct_ones)
print("proportion :", mean, float(correct_ones)/total)

#mean = np.mean(predictedTFIDF == Y_test)
#total = len(Y_test)
#correct_ones = np.sum(predictedTFIDF == Y_test)
#print("total test :", total)
#print("correct :", correct_ones)
#print("proportion :", mean, float(correct_ones)/total)

total test : 73756
correct : 73540
proportion : 0.9970714246976518 0.9970714246976518


In [9]:
import json

ref = {
    "0" : "float",
    "1" : "int",
    "2" : "code postal",
    "3" : "coordonée gps",
    "4" : "adresse",
    "5" : "siret",
    "6" : "année",
    "7" : "date",
    "8" : "siren",
    "9" : "naf",
    "10" : "autre",
    "11" : "téléphone",
}

for i in np.where(predicted != Y_test)[0]:
    print(i, X_test[i], ref[str(pipeline_clf.predict([X_test[i]])[0])], ref[str(Y_test[i])])

584 9 BH24 1HY
 adresse code postal
691 6 10587
 coordonée gps code postal
972 8 M15 6SE
 adresse code postal
1134 7 014459
 téléphone code postal
1153 9 SO53 3LD
 adresse code postal
1332 6 49066
 coordonée gps code postal
1450 9 19000101
 code postal date
1466 9 19960110
 code postal date
1652 9 19800430
 code postal date
2444 8 HG3 IPY
 adresse code postal
2924 9 19800415
 code postal date
2986 5 5032
 siret code postal
3606 9 19821110
 code postal date
4484 9 CV21 2NH
 adresse code postal
5267 10 Co-Dublin
 adresse code postal
5422 8 SP1 2LN
 adresse code postal
5527 6 44815
 coordonée gps code postal
6018 9 19921001
 code postal date
6312 8 2333 CH
 adresse code postal
7100 9 20000313
 code postal date
7350 8 RG2 6AD
 adresse code postal
7472 8 2333 CR
 adresse code postal
7494 8 8243 RC
 adresse code postal
7515 9 MK16 ODG
 adresse code postal
8102 8 5653 ma
 adresse code postal
8130 9 19951121
 coordonée gps date
8380 6 44321
 coordonée gps code postal
8899 5 4000
 siret code po

In [ ]:
text = input("saisissez du text : ")
lang = pipeline_clf.predict([text])
print(lang)